In [ ]:
# default_exp knots

# Module knot

> Here we have the Nodes wrapping around our core functions, which can be added to a Nipype workflow to process your input images. These definitions here are our custom functions and Nipype Nodes knotted together, which creates a workflow.

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
%%capture
#export
import os

from pipetography.core import *

from nipype import IdentityInterface, Function
from nipype.interfaces.io import SelectFiles
from nipype.pipeline import Node

Get images per subject with Nipype Node:

In [ ]:
#export
def BIDS_input(subject_list, sub_dir = 'data'):
    """
    Create input data source as Nipype nodes
    Arguments:
        subjects_list : list of subjects in BIDS directory, output of `get_subs` function
        sub_dir       : path to BIDS folder with subjects sub-folders
    Returns:
        sub_source : Nipype node of subject iterables
        selectfiles: Nipype node of iterated subject's files
    """
    # create iterable that sweeps all subjects:
    sub_source = Node(IdentityInterface(fields = ['subject_id']), name = 'data_source')
    sub_source.iterables = [('subject_id', subject_list)]
    
    # Select files based on BIDS structure template:
    dwi_file = os.path.join('sub-{subject_id}', 'ses-*', 'dwi', 'sub-{subject_id}_ses-*_dwi.nii.gz')
    bv_files = os.path.join('sub-{subject_id}', 'ses-1', 'dwi', 'sub-{subject_id}_ses-1_dwi.bv*')
    templates = {'dwi': dwi_file, 'bvs': bv_files}
    # then create Node:
    select_files = Node(SelectFiles(templates, base_directory = sub_dir), name ='selectfiles')
    
    return sub_source, select_files

In [ ]:
#example usage:
sub_list = get_subs() # from pipetography.core, defaults to search 'data' folder
sub_source, select_files = select_BIDS_files(subject_list = sub_list)

Creating layout of data directory, might take a while if there are a lot of subjects


Find `.bval/.bvec` files with Nipype Node:

In [ ]:
#export
def bfiles_input(BIDS_layout):
    """
    Create Nipype node for bfiles data input
    """
    bfiles_getter = Node(Function(input_names=['BIDS_layout'], output_names=['out_path'], function = get_bfiles), name = 'get_bfiles')
    
    return bfiles_getter

In [ ]:
#example usage:


Nodes for preprocessing steps:

Node for atlas selection:

In [ ]:
#export
def atlas_input(atlas_dir, atlas_names):
    """
    Create Nipype node for selecting atlas images in `atlas_dir`, the node will iterate over `atlas_names`
    Arguments:
        atlas_dir (str): path to folder containing atlas volumes
        atlas_names (List): List of strings matching the file names of each atlas you'd like to use
    """
    atlas_source = Node(IdentityInterface(fields = ['atlas_name']), name = 'atlas_source')
    atlas_source.iterables = [('atlas_name', atlas_names)]
    atlas_template = {'atlas': atlas_dir + '{file_name}'}
    select_atlas = Node(SelectFiles(template), name = 'select_atlas')
    select_atlas.base_directory = atlas_dir
    
    return atlas_source, select_atlas

Node for connecting mask volume

In [ ]:
#export
def mask_input():
    
    return mask_tuple_getter

A `pipeline` class that knots everything together.

In [ ]:
#export
class pipeline():
    
    